<a href="https://colab.research.google.com/github/Alvaro-Garcia-Parra/AA_GRUPO3/blob/main/Lab04_Pr4PUBG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías y funciones a utilizar

In [48]:
import mlxtend
from mlxtend.frequent_patterns import association_rules, apriori
from mlxtend import frequent_patterns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable

# Objetivo
Bluehole, la empresa encargada del videojuego PlayerUnknown’s Battlegrounds quiere introducir nuevos
paquetes dependiendo del tipo de jugador. Para ello dispone de estadísticas de los 200 mejores jugadores.Aplica un algoritmo de manera que se obtengan dichos grupos.

1) Utiliza varias configuraciones teniendo en cuenta el número de grupos que se creará y cambiando cómo
se mide la distancia entre individuos. Crea una tabla donde se incluya toda la información y el número
necesario de iteraciones para llegar a dicha solución. Se considera la mejor solución aquella que necesite
menos iteraciones.

2) Con la mejor configuración del punto anterior. Utiliza dos criterios para elegir el lugar inicial del
punto central de los grupos. Dibuja cómo se van modificando los grupos y cómo van cambiando
sus centroides en cada iteración. Obtén una conclusión acerca de dónde deberían situarse los
centroides.

3) Estudia qué técnicas de preprocesamiento se podrían incluir en base al error cometido en cada
cluster.

## Desarrollo
### Lectura y preprocesamiento de los datos
No hay datos vacíos ni duplicados

In [49]:
# Lectura de la base de datos
filename = 'PUBG.csv'
data = pd.read_csv(filename, sep = ',')

# Revisión de los tipos de datos
print(data.dtypes)
data

player_name             object
tracker_id               int64
solo_KillDeathRatio    float64
solo_WinRatio          float64
solo_TimeSurvived      float64
                        ...   
squad_Heals              int64
squad_Revives            int64
squad_Boosts             int64
squad_DamageDealt      float64
squad_DBNOs              int64
Length: 152, dtype: object


,player_name,tracker_id,solo_KillDeathRatio,solo_WinRatio,solo_TimeSurvived,solo_RoundsPlayed,solo_Wins,solo_WinTop10Ratio,solo_Top10s,solo_Top10Ratio,...,squad_RideDistance,squad_MoveDistance,squad_AvgWalkDistance,squad_AvgRideDistance,squad_LongestKill,squad_Heals,squad_Revives,squad_Boosts,squad_DamageDealt,squad_DBNOs
0,BreakNeck,4405,3.14,17.65,18469.14,17,3,0.83,4,23.5,...,3751590.99,5194786.58,2626.97,4372.64,536.98,2186,234,1884,242132.73,1448
1,Blackwalk,8199,4.41,18.18,33014.86,33,6,0.36,11,33.3,...,4295917.30,6051783.67,2422.48,6009.73,734.85,2041,276,2340,269795.75,1724
2,mercedes_benz,4454,3.60,0.00,4330.44,5,0,0.00,1,20.0,...,3935265.63,5589608.74,1871.89,3011.87,725.44,1766,210,2193,292977.07,1897
3,DORA,7729,14.00,50.00,13421.82,8,4,0.67,6,75.0,...,2738998.00,3796916.00,2154.62,5578.41,587.28,1214,142,1252,181106.90,1057
4,n2tstar,0,10.50,33.33,9841.04,6,2,0.40,5,83.3,...,2347295.00,3220260.00,2098.47,5642.54,546.10,1245,120,923,160029.80,1077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,jungwook,11690,3.37,17.33,581263.40,427,74,0.38,195,45.7,...,585880.94,1010687.19,2015.37,2582.05,796.52,520,71,513,69526.76,456
95,Sitton,6042,6.00,27.27,15639.28,11,3,0.60,5,45.5,...,1370085.00,2075036.00,2049.28,3982.80,511.08,823,109,891,97551.31,573
96,PANDATV-TYF,22188,3.05,9.84,243659.10,193,19,0.25,77,39.9,...,1079497.60,1870113.40,1750.42,2107.95,560.26,998,166,741,135111.34,856
97,Great_Cornholio,2260,0.00,0.00,561.04,1,0,0.00,0,0.0,...,1280647.63,2002011.90,1848.11,1898.83,730.65,1088,74,573,119974.01,754


In [50]:
data = data.drop_duplicates()
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 0 to 98
Data columns (total 152 columns):
 #    Column                     Non-Null Count  Dtype  
---   ------                     --------------  -----  
 0    player_name                99 non-null     object 
 1    tracker_id                 99 non-null     int64  
 2    solo_KillDeathRatio        99 non-null     float64
 3    solo_WinRatio              99 non-null     float64
 4    solo_TimeSurvived          99 non-null     float64
 5    solo_RoundsPlayed          99 non-null     int64  
 6    solo_Wins                  99 non-null     int64  
 7    solo_WinTop10Ratio         99 non-null     float64
 8    solo_Top10s                99 non-null     int64  
 9    solo_Top10Ratio            99 non-null     float64
 10   solo_Losses                99 non-null     int64  
 11   solo_Rating                99 non-null     float64
 12   solo_BestRating            99 non-null     float64
 13   solo_DamagePg              99 non-n

<ipython-input-50-b6607a5c476f>:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  data.info(verbose=True, null_counts=True)


In [51]:
data = data.drop(data.columns[[0, 1]], axis = 1)
data

,solo_KillDeathRatio,solo_WinRatio,solo_TimeSurvived,solo_RoundsPlayed,solo_Wins,solo_WinTop10Ratio,solo_Top10s,solo_Top10Ratio,solo_Losses,solo_Rating,...,squad_RideDistance,squad_MoveDistance,squad_AvgWalkDistance,squad_AvgRideDistance,squad_LongestKill,squad_Heals,squad_Revives,squad_Boosts,squad_DamageDealt,squad_DBNOs
0,3.14,17.65,18469.14,17,3,0.83,4,23.5,14,1559.78,...,3751590.99,5194786.58,2626.97,4372.64,536.98,2186,234,1884,242132.73,1448
1,4.41,18.18,33014.86,33,6,0.36,11,33.3,27,1884.53,...,4295917.30,6051783.67,2422.48,6009.73,734.85,2041,276,2340,269795.75,1724
2,3.60,0.00,4330.44,5,0,0.00,1,20.0,5,1255.92,...,3935265.63,5589608.74,1871.89,3011.87,725.44,1766,210,2193,292977.07,1897
3,14.00,50.00,13421.82,8,4,0.67,6,75.0,4,1798.70,...,2738998.00,3796916.00,2154.62,5578.41,587.28,1214,142,1252,181106.90,1057
4,10.50,33.33,9841.04,6,2,0.40,5,83.3,4,1668.67,...,2347295.00,3220260.00,2098.47,5642.54,546.10,1245,120,923,160029.80,1077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,3.37,17.33,581263.40,427,74,0.38,195,45.7,353,2952.00,...,585880.94,1010687.19,2015.37,2582.05,796.52,520,71,513,69526.76,456
95,6.00,27.27,15639.28,11,3,0.60,5,45.5,8,1754.57,...,1370085.00,2075036.00,2049.28,3982.80,511.08,823,109,891,97551.31,573
96,3.05,9.84,243659.10,193,19,0.25,77,39.9,174,2624.78,...,1079497.60,1870113.40,1750.42,2107.95,560.26,998,166,741,135111.34,856
97,0.00,0.00,561.04,1,0,0.00,0,0.0,1,1201.09,...,1280647.63,2002011.90,1848.11,1898.83,730.65,1088,74,573,119974.01,754


In [52]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# crear una lista con diferentes valores del número de grupos y medidas de distancia
n_clusters_list = [2, 3, 4, 5]
metric_list = ['euclidean', 'manhattan', 'cosine']

# Creación de la tabla
tabla = PrettyTable()
tabla.field_names = ["Cluster","Distance","Iterations","Score"]


# iterar sobre los valores de n_clusters y metric
for n_clusters in n_clusters_list:
    for metric in metric_list:
        
        # ajustar el modelo kmeans
        kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10, max_iter=300, tol=1e-4, random_state=42, algorithm = 'auto')
        kmeans.fit(data)
        
        # calcular el número de iteraciones necesarias para llegar a la convergencia
        iterations = kmeans.n_iter_
        
        # calcular el score
        score = silhouette_score(data, kmeans.labels_, metric = metric)

        # añadir los resultados a la lista
        tabla.add_row([n_clusters, metric, iterations, round(score,4)])

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:1366: FutureWarning: algorithm='auto' is deprecated, it will be removed in 1.3. Using 'lloyd' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:1366: FutureWarning: algorithm='auto' is deprecated, it will be removed in 1.3. Using 'lloyd' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:1366: FutureWarning: algorithm='auto' is deprecated, it will be removed in 1.3. Using 'lloyd' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:1366: FutureWarning: algorithm='auto' is deprecated, it will be removed in 1.3. Using 'lloyd' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:1366: FutureWarning: algorithm='auto' is deprecated, it will be removed in 1.3. Using 'lloyd' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:1366

In [53]:
# Mostramos los resultados
tabla.sortby = "Score"
tabla.reversesort = True
print(tabla)

+---------+-----------+------------+---------+
| Cluster |  Distance | Iterations |  Score  |
+---------+-----------+------------+---------+
|    3    | euclidean |     3      |  0.4429 |
|    4    | euclidean |     7      |  0.4368 |
|    5    | euclidean |     3      |  0.4357 |
|    2    | euclidean |     9      |  0.4354 |
|    3    | manhattan |     3      |  0.4126 |
|    2    | manhattan |     9      |  0.395  |
|    4    | manhattan |     7      |  0.3806 |
|    5    | manhattan |     3      |  0.3768 |
|    3    |   cosine  |     3      |  0.1891 |
|    2    |   cosine  |     9      |  0.131  |
|    4    |   cosine  |     7      |  -0.144 |
|    5    |   cosine  |     3      | -0.1575 |
+---------+-----------+------------+---------+
